In [1]:
#6/25/2021 - Roadmap - 1. build in playing untrained and trained agent functionality 2. stress test everything 3. send to compstrat professors / professor zaman if they are interested in using it in a class




!pip install tensorboardX

import gym
from datetime import date

import pandas as pd


import random
import torch
import torch.nn as nn
import torch.nn.utils as nn_utils
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy
import math
from gym import spaces, logger

import sys
import os
import time
import argparse
from tensorboardX import SummaryWriter
import tensorflow
import datetime
import collections

from collections import namedtuple, deque


%load_ext tensorboard


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.0 MB/s eta 0:00:00


In [2]:
GAMMA = 0.99
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
REPLAY_SIZE = 100000
REPLAY_INITIAL = 10000
REWARD_STEPS = 5
EPSILON = 0.3
NUM_ENVS = 30
LEARNING_RATE_ACTOR = 0.001
LEARNING_RATE_CRITIC = 0.001
STOP_REWARD = 200000

TEST_ITERS = 1000

VMAX = 2000
VMIN = 0
N_ATOMS = 4001
DELTA_Z = (VMAX - VMIN) / (N_ATOMS - 1)

HID_SIZE = 64


device = torch.device("cuda")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
class RewardTracker:
    def __init__(self, writer, stop_reward):
        self.writer = writer
        self.stop_reward = stop_reward

    def __enter__(self):
        self.ts = time.time()
        self.ts_frame = 0
        self.total_rewardsA = []
        self.total_rewardsB = []
        self.total_rewardsC = []
        return self

    def __exit__(self, *args):
        self.writer.close()

    def rewardA(self, rewardA, frame, epsilon=None):
        self.total_rewardsA.append(rewardA)
        speed = (frame - self.ts_frame) / (time.time() - self.ts)
        self.ts_frame = frame
        self.ts = time.time()
        mean_rewardA = np.mean(self.total_rewardsA[-100:])
        epsilon_str = "" if epsilon is None else ", eps %.2f" % epsilon
        #print("%d: done %d games, Arthur's mean reward %.3f, Bonnie's mean reward %.3f, Clyde's mean reward %.3f, speed %.2f f/s%s" % (
            #frame, len(self.total_rewardsA), mean_rewardA, mean_rewardB, mean_rewardC, speed, epsilon_str
        #))
        sys.stdout.flush()
        if epsilon is not None:
            self.writer.add_scalar("epsilon", epsilon, frame)
        self.writer.add_scalar("speed", speed, frame)
        self.writer.add_scalar("Arthur's reward_100", mean_rewardA, frame)
        self.writer.add_scalar("Arthur's reward", rewardA, frame)

        if mean_rewardA > self.stop_reward:
            print("Arthur Solved in %d frames!" % frame)
            return True
        return False

    def rewardB(self, rewardB, frame, epsilon=None):

        self.total_rewardsB.append(rewardB)
        speed = (frame - self.ts_frame) / (time.time() - self.ts)
        self.ts_frame = frame
        self.ts = time.time()
        mean_rewardB = np.mean(self.total_rewardsB[-100:])
        epsilon_str = "" if epsilon is None else ", eps %.2f" % epsilon
        #print("%d: done %d games, Arthur's mean reward %.3f, Bonnie's mean reward %.3f, Clyde's mean reward %.3f, speed %.2f f/s%s" % (
            #frame, len(self.total_rewardsA), mean_rewardA, mean_rewardB, mean_rewardC, speed, epsilon_str
        #))
        sys.stdout.flush()
        if epsilon is not None:
            self.writer.add_scalar("epsilon", epsilon, frame)
        self.writer.add_scalar("speed", speed, frame)
        self.writer.add_scalar("Bonnie's reward_100", mean_rewardB, frame)
        self.writer.add_scalar("Bonie's reward", rewardB, frame)

        if mean_rewardB > self.stop_reward:
            print("Bonnie Solved in %d frames!" % frame)
            return True
        return False

    def rewardC(self, rewardC, frame, epsilon=None):

        self.total_rewardsC.append(rewardC)
        speed = (frame - self.ts_frame) / (time.time() - self.ts)
        self.ts_frame = frame
        self.ts = time.time()
        mean_rewardC = np.mean(self.total_rewardsC[-100:])
        epsilon_str = "" if epsilon is None else ", eps %.2f" % epsilon
        #print("%d: done %d games, Arthur's mean reward %.3f, Bonnie's mean reward %.3f, Clyde's mean reward %.3f, speed %.2f f/s%s" % (
            #frame, len(self.total_rewardsA), mean_rewardA, mean_rewardB, mean_rewardC, speed, epsilon_str
        #))
        sys.stdout.flush()
        if epsilon is not None:
            self.writer.add_scalar("epsilon", epsilon, frame)
        self.writer.add_scalar("speed", speed, frame)
        self.writer.add_scalar("Clyde's reward_100", mean_rewardC, frame)
        self.writer.add_scalar("Clyde's reward", rewardC, frame)

        if mean_rewardC > self.stop_reward:
            print("Clyde Solved in %d frames!" % frame)
            return True
        return False

class TBMeanTracker:
    """
    TensorBoard value tracker: allows to batch fixed amount of historical values and write their mean into TB
    Designed and tested with pytorch-tensorboard in mind
    """
    def __init__(self, writer, batch_size):
        """
        :param writer: writer with close() and add_scalar() methods
        :param batch_size: integer size of batch to track
        """
        assert isinstance(batch_size, int)
        assert writer is not None
        self.writer = writer
        self.batch_size = batch_size

    def __enter__(self):
        self._batches = collections.defaultdict(list)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.writer.close()

    @staticmethod
    def _as_float(value):
        assert isinstance(value, (float, int, np.ndarray, np.generic, torch.autograd.Variable)) or torch.is_tensor(value)
        tensor_val = None
        if isinstance(value, torch.autograd.Variable):
            tensor_val = value.data
        elif torch.is_tensor(value):
            tensor_val = value

        if tensor_val is not None:
            return tensor_val.float().mean().item()
        elif isinstance(value, np.ndarray):
            return float(np.mean(value))
        else:
            return float(value)

    def track(self, param_name, value, iter_index):
        assert isinstance(param_name, str)
        assert isinstance(iter_index, int)

        data = self._batches[param_name]
        data.append(self._as_float(value))

        if len(data) >= self.batch_size:
            self.writer.add_scalar(param_name, np.mean(data), iter_index)
            data.clear()

In [4]:
class ShrimpEnv(gym.Env):

     def __init__(self):






            #self.action_space = gym.spaces.Box(low=0,high=100, shape=(1,), dtype=np.float32)
            #self.action_space = gym.spaces.Discrete(100)
            self.action_space = gym.spaces.Discrete(1)
            self.observation_space = gym.spaces.Box(low=0, high=100,shape=(4,), dtype=np.float32)



            self.state = None
            self.reward = None
            self.steps_left = None
            self.done = False


     def step(self, actionA, actionB, actionC):

            self.steps_left -= 1



            done = bool(
                self.steps_left == 1
            )
            #print("action A", actionA)
            actionAs = (actionA[0]*29.3)+50
            actionBs = (actionB[0]*29.3)+50
            actionCs = (actionC[0]*29.3)+50

            self.raw_price = 100-(0.5*(actionAs+actionBs+actionCs))

            if self.raw_price <= 0:

              self.todays_price = 0

            else:

              self.todays_price = self.raw_price


            self.a_profit = actionAs*(self.todays_price-10)

            self.b_profit = actionBs*(self.todays_price-10)

            self.c_profit = actionCs*(self.todays_price-10)

            self.state = np.array([actionAs, actionBs, actionCs, self.todays_price])

            self.reward = self.a_profit, self.b_profit, self.c_profit

            #print("end step state", self.state)


            return np.transpose(self.state), self.reward, done, {}

     def reset(self):


        self.steps_left = 2


        self.state = np.array((0,0,0,0),dtype=np.float32)

        #print("reset state", self.state)

        self.reward = 0

        self.done = False

        return np.transpose(self.state)

In [5]:
#NN Models

class ShrimpActor(nn.Module):
    def __init__(self, obs_size, act_size):
        super(ShrimpActor, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(obs_size, 400),
            nn.ReLU(),
            nn.Linear(400, 300),
            nn.ReLU(),
            nn.Linear(300, act_size),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)



class ShrimpCritic(nn.Module):
    def __init__(self, obs_size, act_size,
                 n_atoms, v_min, v_max):
        super(ShrimpCritic, self).__init__()

        self.obs_net = nn.Sequential(
            nn.Linear(obs_size, 400),
            nn.ReLU(),
        )

        self.out_net = nn.Sequential(
            nn.Linear(400 + act_size, 300),
            nn.ReLU(),
            nn.Linear(300, n_atoms)
        )

        delta = (v_max - v_min) / (n_atoms - 1)
        self.register_buffer("supports", torch.arange(
            v_min, v_max + delta, delta))

    def forward(self, x, a):
        obs = self.obs_net(x)
        return self.out_net(torch.cat([obs, a], dim=1))

    def distr_to_q(self, distr):
        weights = F.softmax(distr, dim=1) * self.supports
        res = weights.sum(dim=1)
        return res.unsqueeze(dim=-1)

In [6]:
class BaseAgent:
    """
    Abstract Agent interface
    """
    def initial_state(self):
        """
        Should create initial empty state for the agent. It will be called for the start of the episode
        :return: Anything agent want to remember
        """
        return None

    def __call__(self, states, agent_states):
        """
        Convert observations and states into actions to take
        :param states: list of environment states to process
        :param agent_states: list of states with the same length as observations
        :return: tuple of actions, states
        """
        assert isinstance(states, list)
        assert isinstance(agent_states, list)
        assert len(agent_states) == len(states)

        raise NotImplementedError

def float32_preprocessor(states):
    np_states = np.array(states, dtype=np.float32)
    return torch.tensor(np_states)


class AgentD4PG(BaseAgent):
    """
    Agent implementing noisy agent
    """
    def __init__(self, net, device=device, epsilon=EPSILON):
        self.net = net
        self.device = device
        self.epsilon = epsilon

    def __call__(self, states, agent_states):
        states_v = float32_preprocessor(states)
        states_v = states_v.to(self.device)
        mu_v = self.net(states_v)
        actions = mu_v.data.cpu().numpy()
        actions += self.epsilon * np.random.normal(
            size=actions.shape)
        #actions = np.clip(actions, -1.7, 1.7)
        return actions, agent_states


class InsiderAgentD4PG(BaseAgent):
    """
    Agent class to inject inside information
    """
    def __init__(self, net, device=device, epsilon=EPSILON):
        self.net = net
        self.device = device
        self.epsilon = epsilon

    def __call__(self, states, agent_states):
        states_v = float32_preprocessor(states)
        states_v = states_v.to(self.device)
        mu_v = self.net(states_v)


        actions = mu_v.data.cpu().numpy()
        actions += self.epsilon * np.random.normal(size=actions.shape)
        #actions = np.clip(actions, -1.7, 1.7)

        if random.uniform(0,1) <= .5:

            for env in range(NUM_ENVS):

              actions[env] = (45-50)/29.3

        return actions, agent_states

class TargetNet:
    """
    Wrapper around model which provides copy of it instead of trained weights
    """
    def __init__(self, model):
        self.model = model
        self.target_model = copy.deepcopy(model)

    def sync(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def alpha_sync(self, alpha):
        """
        Blend params of target net with params from the model
        :param alpha:
        """
        assert isinstance(alpha, float)
        assert 0.0 < alpha <= 1.0
        state = self.model.state_dict()
        tgt_state = self.target_model.state_dict()
        for k, v in state.items():
            tgt_state[k] = tgt_state[k] * alpha + (1 - alpha) * v
        self.target_model.load_state_dict(tgt_state)


In [7]:
# one single experience step
#Experience = namedtuple('Experience', ['state', 'action', 'reward', 'done'])
Experience = namedtuple('Experience', ['state', 'actionA', 'actionB', 'actionC', 'rewardA', 'rewardB', 'rewardC','done'])


class ExperienceSourceMARL:
    """
    Simple n-step experience source using single or multiple environments
    Every experience contains n list of Experience entries
    """
    def __init__(self, env, agentA, agentB, agentC, steps_count=2, steps_delta=1, vectorized=False):
        """
        Create simple experience source
        :param env: environment or list of environments to be used
        :param agent: callable to convert batch of states into actions to take
        :param steps_count: count of steps to track for every experience chain
        :param steps_delta: how many steps to do between experience items
        :param vectorized: support of vectorized envs from OpenAI universe
        """
        assert isinstance(env, (gym.Env, list, tuple))
        assert isinstance(agentA, BaseAgent)
        assert isinstance(agentB, BaseAgent)
        assert isinstance(agentC, BaseAgent)
        assert isinstance(steps_count, int)
        assert steps_count >= 1
        assert isinstance(vectorized, bool)
        if isinstance(env, (list, tuple)):
            self.pool = env
        else:
            self.pool = [env]
        self.agentA = agentA
        self.agentB = agentB
        self.agentC = agentC
        self.steps_count = steps_count
        self.steps_delta = steps_delta
        self.total_rewardsA = []
        self.total_rewardsB = []
        self.total_rewardsC = []
        self.total_steps = []
        self.vectorized = vectorized

    def __iter__(self):
        states, agent_states, histories, cur_rewardsA, cur_rewardsB, cur_rewardsC, cur_steps = [], [], [], [], [], [], []
        env_lens = []
        for env in self.pool:
            obs = env.reset()
            print(type(obs))
            # if the environment is vectorized, all it's output is lists of results.
            # Details are here: https://github.com/openai/universe/blob/master/doc/env_semantics.rst
            if self.vectorized:
                obs_len = len(obs)
                states.extend(obs)
            else:
                obs_len = 1
                states.append(obs)
            env_lens.append(obs_len)

            for _ in range(obs_len):
                histories.append(deque(maxlen=self.steps_count))
                cur_rewardsA.append(0.0)
                cur_rewardsB.append(0.0)
                cur_rewardsC.append(0.0)
                cur_steps.append(0)
                #In shrimp game all agents share the same states
                agent_states.append(self.agentA.initial_state())

        iter_idx = 0
        while True:
            actionsA = [None] * len(states)
            actionsB = [None] * len(states)
            actionsC = [None] * len(states)
            states_input = []
            states_indices = []
            for idx, state in enumerate(states):
                if state is None:
                    actionsA[idx] = self.pool[0].action_space.sample()  # assume that all envs are from the same family
                    actionsB[idx] = self.pool[0].action_space.sample()  # assume that all envs are from the same family
                    actionsC[idx] = self.pool[0].action_space.sample()  # assume that all envs are from the same family
                else:
                    states_input.append(state)
                    states_indices.append(idx)
            if states_input:

                states_actionsA, new_agent_states = self.agentA(states_input, agent_states)
                states_actionsB = self.agentB(states_input, agent_states)[0]
                states_actionsC = self.agentC(states_input, agent_states)[0]


                #agent actions
                for idx, (actionA, actionB, actionC) in enumerate(zip(states_actionsA, states_actionsB, states_actionsC)):
                    g_idx = states_indices[idx]
                    actionsA[g_idx] = actionA
                    actionsB[g_idx] = actionB
                    actionsC[g_idx] = actionC
                    agent_states[g_idx] = new_agent_states[idx]
                grouped_actionsA = _group_list(actionsA, env_lens)
                grouped_actionsB = _group_list(actionsB, env_lens)
                grouped_actionsC = _group_list(actionsC, env_lens)

            global_ofs = 0
            for env_idx, (env, action_A, action_B, action_C) in enumerate(zip(self.pool, grouped_actionsA, grouped_actionsB, grouped_actionsC)):
                if self.vectorized:
                    next_state_n, r_n, is_done_n, _ = env.step(action_A, action_B, action_C)
                else:

                    next_state, r, is_done, _ = env.step(action_A[0], action_B[0], action_C[0])
                    next_state_n, r_n, is_done_n = [next_state], [r], [is_done]

                for ofs, (actionA, actionB, actionC, next_state, r, is_done) in enumerate(zip(action_A, action_B, action_C, next_state_n, r_n, is_done_n)):
                    idx = global_ofs + ofs
                    state = states[idx]
                    history = histories[idx]
                    cur_rewardsA[idx] += r[0]
                    cur_rewardsB[idx] += r[1]
                    cur_rewardsC[idx] += r[2]
                    cur_steps[idx] += 1
                    if state is not None:
                        history.append(Experience(state=state, actionA=actionA, actionB=actionB, actionC=actionC, rewardA=r[0], rewardB=r[1], rewardC=r[2], done=is_done))
                    if len(history) == self.steps_count and iter_idx % self.steps_delta == 0:
                        yield tuple(history)
                    states[idx] = next_state
                    if is_done:
                        # in case of very short episode (shorter than our steps count), send gathered history
                        if 0 < len(history) < self.steps_count:
                            yield tuple(history)
                        # generate tail of history
                        while len(history) > 1:
                            history.popleft()
                            yield tuple(history)

                        #Arthur rewards
                        self.total_rewardsA.append(cur_rewardsA[idx])
                        self.total_steps.append(cur_steps[idx])
                        cur_rewardsA[idx] = 0.0
                        cur_steps[idx] = 0

                        #Bonnie rewards
                        self.total_rewardsB.append(cur_rewardsB[idx])
                        cur_rewardsB[idx] = 0.0

                        #Clyde rewards
                        self.total_rewardsC.append(cur_rewardsC[idx])
                        cur_rewardsC[idx] = 0.0

                        # vectorized envs are reset automatically
                        states[idx] = env.reset() if not self.vectorized else None
                        agent_states[idx] = self.agentA.initial_state()
                        history.clear()
                global_ofs += len(action_A)
            iter_idx += 1

    def pop_total_rewardsA(self):
        r = self.total_rewardsA
        if r:
            self.total_rewardsA = []
            self.total_steps = []
        return r

    def pop_total_rewardsB(self):
        r = self.total_rewardsB
        if r:
            self.total_rewardsB = []
            self.total_steps = []
        return r

    def pop_total_rewardsC(self):
        r = self.total_rewardsC
        if r:
            self.total_rewardsC = []
            self.total_steps = []
        return r

    def pop_rewards_stepsA(self):
        res = list(zip(self.total_rewardsA, self.total_steps))
        if res:
            self.total_rewardsA, self.total_steps = [], []
        return res

    def pop_rewards_stepsB(self):
        res = list(zip(self.total_rewardsB, self.total_steps))
        if res:
            self.total_rewardsB, self.total_steps = [], []
        return res

    def pop_rewards_stepsC(self):
        res = list(zip(self.total_rewardsC, self.total_steps))
        if res:
            self.total_rewardsC, self.total_steps = [], []
        return res



def _group_list(items, lens):
    """
    Unflat the list of items by lens
    :param items: list of items
    :param lens: list of integers
    :return: list of list of items grouped by lengths
    """
    res = []
    cur_ofs = 0
    for g_len in lens:
        res.append(items[cur_ofs:cur_ofs+g_len])
        cur_ofs += g_len
    return res


# those entries are emitted from ExperienceSourceFirstLast. Reward is discounted over the trajectory piece
ExperienceFirstLast = collections.namedtuple('ExperienceFirstLast', ('state', 'actionA', 'actionB', 'actionC', 'rewardA', 'rewardB', 'rewardC', 'last_state'))


class ExperienceSourceFirstLastMARL(ExperienceSourceMARL):
    """
    This is a wrapper around ExperienceSource to prevent storing full trajectory in replay buffer when we need
    only first and last states. For every trajectory piece it calculates discounted reward and emits only first
    and last states and action taken in the first state.
    If we have partial trajectory at the end of episode, last_state will be None
    """
    def __init__(self, env, agentA, agentB, agentC, gamma, steps_count=1, steps_delta=1, vectorized=False):
        assert isinstance(gamma, float)
        super(ExperienceSourceFirstLastMARL, self).__init__(env, agentA, agentB, agentC, steps_count+1, steps_delta, vectorized=vectorized)
        self.gamma = gamma
        self.steps = steps_count

    def __iter__(self):
        for exp in super(ExperienceSourceFirstLastMARL, self).__iter__():
            if exp[-1].done and len(exp) <= self.steps:
                last_state = None
                elems = exp
            else:
                last_state = exp[-1].state
                elems = exp[:-1]
            total_rewardA = 0.0
            total_rewardB = 0.0
            total_rewardC = 0.0

            for e in reversed(elems):
                total_rewardA *= self.gamma
                total_rewardA += e.rewardA

                total_rewardB *= self.gamma
                total_rewardB += e.rewardB

                total_rewardC *= self.gamma
                total_rewardC += e.rewardC
            yield ExperienceFirstLast(state=exp[0].state, actionA=exp[0].actionA, actionB=exp[0].actionB, actionC=exp[0].actionC,
                                      rewardA=total_rewardA, rewardB=total_rewardB, rewardC=total_rewardC,last_state=last_state)

class ExperienceReplayBuffer:
    def __init__(self, experience_source, buffer_size):
        assert isinstance(experience_source, (ExperienceSourceMARL, type(None)))
        assert isinstance(buffer_size, int)
        self.experience_source_iter = None if experience_source is None else iter(experience_source)
        self.buffer = []
        self.capacity = buffer_size
        self.pos = 0

    def __len__(self):
        return len(self.buffer)

    def __iter__(self):
        return iter(self.buffer)

    def sample(self, batch_size):
        """
        Get one random batch from experience replay
        TODO: implement sampling order policy
        :param batch_size:
        :return:
        """
        if len(self.buffer) <= batch_size:
            return self.buffer
        # Warning: replace=False makes random.choice O(n)
        keys = np.random.choice(len(self.buffer), batch_size, replace=True)
        return [self.buffer[key] for key in keys]

    def _add(self, sample):
        if len(self.buffer) < self.capacity:
            self.buffer.append(sample)
        else:
            self.buffer[self.pos] = sample
        self.pos = (self.pos + 1) % self.capacity

    def populate(self, samples):
        """
        Populates samples into the buffer
        :param samples: how many samples to populate
        """
        for _ in range(samples):
            entry = next(self.experience_source_iter)
            self._add(entry)

class PrioritizedReplayBuffer(ExperienceReplayBuffer):
    def __init__(self, experience_source, buffer_size, alpha):
        super(PrioritizedReplayBuffer, self).__init__(experience_source, buffer_size)
        assert alpha > 0
        self._alpha = alpha

        it_capacity = 1
        while it_capacity < buffer_size:
            it_capacity *= 2

        self._it_sum = utils.SumSegmentTree(it_capacity)
        self._it_min = utils.MinSegmentTree(it_capacity)
        self._max_priority = 1.0

    def _add(self, *args, **kwargs):
        idx = self.pos
        super()._add(*args, **kwargs)
        self._it_sum[idx] = self._max_priority ** self._alpha
        self._it_min[idx] = self._max_priority ** self._alpha

    def _sample_proportional(self, batch_size):
        res = []
        for _ in range(batch_size):
            mass = random.random() * self._it_sum.sum(0, len(self) - 1)
            idx = self._it_sum.find_prefixsum_idx(mass)
            res.append(idx)
        return res

    def sample(self, batch_size, beta):
        assert beta > 0

        idxes = self._sample_proportional(batch_size)

        weights = []
        p_min = self._it_min.min() / self._it_sum.sum()
        max_weight = (p_min * len(self)) ** (-beta)

        for idx in idxes:
            p_sample = self._it_sum[idx] / self._it_sum.sum()
            weight = (p_sample * len(self)) ** (-beta)
            weights.append(weight / max_weight)
        weights = np.array(weights, dtype=np.float32)
        samples = [self.buffer[idx] for idx in idxes]
        return samples, idxes, weights

    def update_priorities(self, idxes, priorities):
        assert len(idxes) == len(priorities)
        for idx, priority in zip(idxes, priorities):
            assert priority > 0
            assert 0 <= idx < len(self)
            self._it_sum[idx] = priority ** self._alpha
            self._it_min[idx] = priority ** self._alpha

            self._max_priority = max(self._max_priority, priority)

In [15]:
def test_net(netA, netB, netC, env, count=NUM_ENVS, device=device):
    rewardsA = 0.0
    rewardsB = 0.0
    rewardsC = 0.0
    steps = 0
    for _ in range(count):
        obs = env.reset()
        while True:
            obs_v = float32_preprocessor([obs]).to(device)
            mu_vA = netA(obs_v)
            mu_vB = netB(obs_v)
            mu_vC = netC(obs_v)
            actionA = mu_vA.squeeze(dim=0).data.cpu().numpy()
            actionB = mu_vB.squeeze(dim=0).data.cpu().numpy()
            actionC = mu_vC.squeeze(dim=0).data.cpu().numpy()
            #actionA = np.clip(actionA, -1.7, 1.7)
            #actionB = np.clip(actionB, -1.7, 1.7)
            #actionC = np.clip(actionC, -1.7, 1.7)
            obs, reward, done, _ = env.step(actionA, actionB, actionC)
            rewardsA += reward[0]
            rewardsB += reward[1]
            rewardsC += reward[2]
            steps += 1
            if done:
                break
    return rewardsA / count, rewardsB / count, rewardsC / count, steps / count


def unpack_batch_ddqn(batch, device=device):
    states, actionsA, actionsB, actionsC, rewardsA, rewardsB, rewardsC, dones, last_states = [], [], [], [], [], [], [], [], []
    for exp in batch:
        states.append(exp.state)
        actionsA.append(exp.actionA)
        actionsB.append(exp.actionB)
        actionsC.append(exp.actionC)
        rewardsA.append(exp.rewardA)
        rewardsB.append(exp.rewardB)
        rewardsC.append(exp.rewardC)
        dones.append(exp.last_state is None)
        if exp.last_state is None:
            last_states.append(exp.state)
        else:
            last_states.append(exp.last_state)
    states_v = float32_preprocessor(states).to(device)
    actions_vA = float32_preprocessor(actionsA).to(device)
    rewards_vA = float32_preprocessor(rewardsA).to(device)
    actions_vB = float32_preprocessor(actionsB).to(device)
    rewards_vB = float32_preprocessor(rewardsB).to(device)
    actions_vC = float32_preprocessor(actionsC).to(device)
    rewards_vC = float32_preprocessor(rewardsC).to(device)
    last_states_v = float32_preprocessor(last_states).to(device)
    dones_t = torch.BoolTensor(dones).to(device)
    return states_v, actions_vA, actions_vB, actions_vC, rewards_vA, rewards_vB, rewards_vC, dones_t, last_states_v

def distr_projection(next_distr_v, rewards_v, dones_mask_t,
                     gamma, device="cpu"):
    next_distr = next_distr_v.data.cpu().numpy()
    rewards = rewards_v.data.cpu().numpy()
    dones_mask = dones_mask_t.cpu().numpy().astype(bool)
    batch_size = len(rewards)
    proj_distr = np.zeros((batch_size, N_ATOMS), dtype=np.float32)

    for atom in range(N_ATOMS):
        tz_j = np.minimum(VMAX, np.maximum(
            VMIN, rewards + (VMIN + atom * DELTA_Z) * gamma))
        b_j = (tz_j - VMIN) / DELTA_Z
        l = np.floor(b_j).astype(np.int64)
        u = np.ceil(b_j).astype(np.int64)
        eq_mask = u == l
        proj_distr[eq_mask, l[eq_mask]] += \
            next_distr[eq_mask, atom]
        ne_mask = u != l
        proj_distr[ne_mask, l[ne_mask]] += \
            next_distr[ne_mask, atom] * (u - b_j)[ne_mask]
        proj_distr[ne_mask, u[ne_mask]] += \
            next_distr[ne_mask, atom] * (b_j - l)[ne_mask]

    if dones_mask.any():
        proj_distr[dones_mask] = 0.0
        tz_j = np.minimum(VMAX, np.maximum(
            VMIN, rewards[dones_mask]))
        b_j = (tz_j - VMIN) / DELTA_Z
        l = np.floor(b_j).astype(np.int64)
        u = np.ceil(b_j).astype(np.int64)
        eq_mask = u == l
        eq_dones = dones_mask.copy()
        eq_dones[dones_mask] = eq_mask
        if eq_dones.any():
            proj_distr[eq_dones, l[eq_mask]] = 1.0
        ne_mask = u != l
        ne_dones = dones_mask.copy()
        ne_dones[dones_mask] = ne_mask
        if ne_dones.any():
            proj_distr[ne_dones, l[ne_mask]] = (u - b_j)[ne_mask]
            proj_distr[ne_dones, u[ne_mask]] = (b_j - l)[ne_mask]
    return torch.FloatTensor(proj_distr).to(device)

In [14]:
if __name__ == "__main__":

    save_path = os.path.join("saves", "a2c-" + "ShrimpNet")
    os.makedirs(save_path, exist_ok=True)

    envs = [ShrimpEnv() for _ in range(NUM_ENVS)]
    test_env = ShrimpEnv()
    writer = SummaryWriter(comment="ShrimpNet")

    netA_actor = ShrimpActor(envs[0].observation_space.shape[0], envs[0].action_space.n).to(device)
    netA_critic = ShrimpCritic(envs[0].observation_space.shape[0], envs[0].action_space.n, N_ATOMS, VMIN, VMAX).to(device)
    #print(netA_actor)
    #print(netA_critic)
    tgt_netA_actor = TargetNet(netA_actor)
    tgt_netA_critic = TargetNet(netA_critic)

    netB_actor = ShrimpActor(envs[0].observation_space.shape[0], envs[0].action_space.n).to(device)
    netB_critic = ShrimpCritic(envs[0].observation_space.shape[0], envs[0].action_space.n, N_ATOMS, VMIN, VMAX).to(device)

    tgt_netB_actor = TargetNet(netB_actor)
    tgt_netB_critic = TargetNet(netB_critic)

    netC_actor = ShrimpActor(envs[0].observation_space.shape[0], envs[0].action_space.n).to(device)
    netC_critic = ShrimpCritic(envs[0].observation_space.shape[0], envs[0].action_space.n, N_ATOMS, VMIN, VMAX).to(device)

    tgt_netC_actor = TargetNet(netC_actor)
    tgt_netC_critic = TargetNet(netC_critic)

    agentA = AgentD4PG(netA_actor, device=device)

    agentB = AgentD4PG(netB_actor, device=device)

    agentC = AgentD4PG(netC_actor, device=device)

    exp_source = ExperienceSourceFirstLastMARL(envs, agentA, agentB, agentC, gamma=GAMMA, steps_count=REWARD_STEPS)
    buffer = ExperienceReplayBuffer(exp_source, buffer_size=REPLAY_SIZE)

    optimizerA_actor = optim.Adam(netA_actor.parameters(), lr=LEARNING_RATE_ACTOR)
    optimizerA_critic = optim.Adam(netA_critic.parameters(), lr=LEARNING_RATE_CRITIC)

    optimizerB_actor = optim.Adam(netB_actor.parameters(), lr=LEARNING_RATE_ACTOR)
    optimizerB_critic = optim.Adam(netB_critic.parameters(), lr=LEARNING_RATE_CRITIC)

    optimizerC_actor = optim.Adam(netC_actor.parameters(), lr=LEARNING_RATE_ACTOR)
    optimizerC_critic = optim.Adam(netC_critic.parameters(), lr=LEARNING_RATE_CRITIC)

    frame_idx = 0
    best_rewardA = None
    best_rewardB = None
    best_rewardC = None

In [ ]:
with RewardTracker(writer, STOP_REWARD) as tracker:
        with TBMeanTracker(writer, batch_size=10) as tb_tracker:
            while True:
                frame_idx += 1
                buffer.populate(1)

                rewards_stepsA = exp_source.pop_rewards_stepsA()
                rewards_stepsB = exp_source.pop_rewards_stepsB()
                rewards_stepsC = exp_source.pop_rewards_stepsC()

                if rewards_stepsA:
                    rewardsA, steps = zip(*rewards_stepsA)
                    tb_tracker.track("episode_steps", steps[0], frame_idx)
                    tracker.rewardA(rewardsA[0], frame_idx)

                elif rewards_stepsB:
                    rewardsB, steps = zip(*rewards_stepsB)
                    tb_tracker.track("episode_steps", steps[0], frame_idx)
                    tracker.rewardB(rewardsB[0], frame_idx)

                elif rewards_stepsC:
                    rewardsC, steps = zip(*rewards_stepsC)
                    tb_tracker.track("episode_steps", steps[0], frame_idx)
                    tracker.rewardC(rewardsC[0], frame_idx)

                if len(buffer) < REPLAY_INITIAL:
                    continue

                batch = buffer.sample(BATCH_SIZE)
                states_v, actions_vA, actions_vB, actions_vC, rewards_vA, rewards_vB, rewards_vC, dones_mask, last_states_v = unpack_batch_ddqn(batch, device)

                #print("STATES_V", len(states_v), states_v)
                #print("ACTIONS_V", len(actions_vA), actions_vA)
                #print("REWARDS_V", len(rewards_vA),rewards_vA)
                #print("LAST STATES_V", len(last_states_v), last_states_v)

                actions = []
                actions = ((actions_vA*29.3+50), (actions_vB*29.3+50), (actions_vC*29.3+50))
                mean_actions = (torch.mean(actions_vA*29.3+50)), (torch.mean(actions_vB*29.3+50)), (torch.mean(actions_vC*29.3+50))


                # train Arthur critic
                optimizerA_critic.zero_grad()
                crt_distr_vA = netA_critic(states_v, actions_vA)
                last_act_vA = tgt_netA_actor.target_model(last_states_v)
                last_distr_vA = F.softmax(tgt_netA_critic.target_model(last_states_v, last_act_vA), dim=1)
                proj_distr_vA = distr_projection(last_distr_vA, rewards_vA, dones_mask, gamma=GAMMA**REWARD_STEPS, device=device)
                prob_dist_vA = -F.log_softmax(crt_distr_vA, dim=1) * proj_distr_vA
                critic_loss_vA = prob_dist_vA.sum(dim=1).mean()
                critic_loss_vA.backward()
                optimizerA_critic.step()
                tb_tracker.track("Arthur's loss_critic", critic_loss_vA, frame_idx)

                # train Arthur actor
                optimizerA_actor.zero_grad()
                cur_actions_vA = netA_actor(states_v)
                crt_distr_vA = netA_critic(states_v, cur_actions_vA)
                actor_loss_vA = -netA_critic.distr_to_q(crt_distr_vA)
                actor_loss_vA = actor_loss_vA.mean()
                actor_loss_vA.backward()
                optimizerA_actor.step()
                tb_tracker.track("Arthur's loss_actor", actor_loss_vA,
                                 frame_idx)

                tgt_netA_actor.alpha_sync(alpha=1 - 1e-3)
                tgt_netA_critic.alpha_sync(alpha=1 - 1e-3)

                # train Bonnie critic
                optimizerB_critic.zero_grad()
                crt_distr_vB = netB_critic(states_v, actions_vB)
                last_act_vB = tgt_netB_actor.target_model(last_states_v)
                last_distr_vB = F.softmax(tgt_netB_critic.target_model(last_states_v, last_act_vB), dim=1)
                proj_distr_vB = distr_projection(last_distr_vB, rewards_vB, dones_mask, gamma=GAMMA**REWARD_STEPS, device=device)
                prob_dist_vB = -F.log_softmax(crt_distr_vB, dim=1) * proj_distr_vB
                critic_loss_vB = prob_dist_vB.sum(dim=1).mean()
                critic_loss_vB.backward()
                optimizerB_critic.step()
                tb_tracker.track("Bonnie's loss_critic", critic_loss_vB, frame_idx)

                # train Bonnie actor
                optimizerB_actor.zero_grad()
                cur_actions_vB = netB_actor(states_v)
                crt_distr_vB = netB_critic(states_v, cur_actions_vB)
                actor_loss_vB = -netB_critic.distr_to_q(crt_distr_vB)
                actor_loss_vB = actor_loss_vB.mean()
                actor_loss_vB.backward()
                optimizerB_actor.step()
                tb_tracker.track("Bonnie's loss_actor", actor_loss_vB,
                                 frame_idx)

                tgt_netB_actor.alpha_sync(alpha=1 - 1e-3)
                tgt_netB_critic.alpha_sync(alpha=1 - 1e-3)

                # train Clyde critic
                optimizerC_critic.zero_grad()
                crt_distr_vC = netC_critic(states_v, actions_vC)
                last_act_vC = tgt_netC_actor.target_model(last_states_v)
                last_distr_vC = F.softmax(tgt_netC_critic.target_model(last_states_v, last_act_vC), dim=1)
                proj_distr_vC = distr_projection(last_distr_vC, rewards_vC, dones_mask, gamma=GAMMA**REWARD_STEPS, device=device)
                prob_dist_vC = -F.log_softmax(crt_distr_vC, dim=1) * proj_distr_vC
                critic_loss_vC = prob_dist_vC.sum(dim=1).mean()
                critic_loss_vC.backward()
                optimizerC_critic.step()
                tb_tracker.track("Clyde's loss_critic", critic_loss_vC, frame_idx)

                # train Clyde actor
                optimizerC_actor.zero_grad()
                cur_actions_vC = netC_actor(states_v)
                crt_distr_vC = netC_critic(states_v, cur_actions_vC)
                actor_loss_vC = -netC_critic.distr_to_q(crt_distr_vC)
                actor_loss_vC = actor_loss_vC.mean()
                actor_loss_vC.backward()
                optimizerC_actor.step()
                tb_tracker.track("Clydes's loss_actor", actor_loss_vC,
                                 frame_idx)

                tgt_netC_actor.alpha_sync(alpha=1 - 1e-3)
                tgt_netC_critic.alpha_sync(alpha=1 - 1e-3)

                if frame_idx % TEST_ITERS == 0:
                    ts = time.time()
                    rewardsA, rewardsB, rewardsC, steps = test_net(netA_actor, netB_actor, netC_actor, test_env, device=device)
                    community = (rewardsA + rewardsB + rewardsC)/3
                    #print("Mean rewards", torch.mean(rewards_vA), torch.mean(rewards_vB), torch.mean(rewards_vC))
                    #print("actions", actions)
                    #print("mean actions", mean_actions)
                    #print("Test done in %.2f sec, Arthur's reward %.3f, Bonnie's reward %.3f, Clyde's reward %.3f, Community reward %.3f, steps %d" % (
                        #time.time() - ts, rewardsA, rewardsB, rewardsC, community, steps))
                    writer.add_scalar("Arthur's test_reward", rewardsA, frame_idx)
                    writer.add_scalar("Bonnie's test_reward", rewardsB, frame_idx)
                    writer.add_scalar("Clyde's test_reward", rewardsC, frame_idx)
                    writer.add_scalar("Community Average Rewards", community, frame_idx)
                    writer.add_scalar("test_steps", steps, frame_idx)
                    if best_rewardA is None or best_rewardA < rewardsA:
                        if best_rewardA is not None:
                            print("Arthur's Best reward updated: %.3f -> %.3f" % (best_rewardA, rewardsA))
                            name = "Arthur's best_%+.3f_%d.dat" % (rewardsA, frame_idx)
                            fname = os.path.join(save_path, name)
                            torch.save(netA_actor.state_dict(), fname)
                        best_rewardA = rewardsA

                    elif best_rewardB is None or best_rewardB < rewardsB:
                        if best_rewardB is not None:
                            print("Bonnie's Best reward updated: %.3f -> %.3f" % (best_rewardB, rewardsB))
                            name = "Bonnie's best_%+.3f_%d.dat" % (rewardsB, frame_idx)
                            fname = os.path.join(save_path, name)
                            torch.save(netB_actor.state_dict(), fname)
                        best_rewardB = rewardsB

                    elif best_rewardC is None or best_rewardC < rewardsC:
                        if best_rewardC is not None:
                            print("Clyde's Best reward updated: %.3f -> %.3f" % (best_rewardC, rewardsC))
                            name = "Clyde's best_%+.3f_%d.dat" % (rewardsC, frame_idx)
                            fname = os.path.join(save_path, name)
                            torch.save(netC_actor.state_dict(), fname)
                        best_rewardC = rewardsC

In [ ]:
actions = []
actions = (actions_vA[0:10]*29.3+50), (actions_vB[0:10]*29.3+50), (actions_vC[0:10]*29.3+50)
print(actions)


In [ ]:
%tensorboard --logdir runs